In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px

In [46]:
label_df = pd.read_csv('src/raw_data/ground_truth.csv', index_col = False)
label_df = label_df[['file', 'recovery']]
label_df = label_df[label_df['recovery'] != '[]'].reset_index(drop = True)
label_df

,file,recovery
0,00e03657-8e1e-4c8c-a724-1d3c77b48510,"[[2420.9805555555554, 2438.4241666666667], [31..."
1,0a269af2-4455-4897-8a01-308ff137b15b,"[[10045.820833333333, 10165.977777777778]]"
2,0a9816bb-2a0f-44fa-82ad-cdb6e0d0eeaf,"[[1172.6930555555555, 1173.3916666666667], [12..."
3,0ad8ecc0-2995-4ee0-832c-5b5b4fa76f36,"[[2329.0, 3457.0]]"
4,0ae3634c-2236-42bc-94c4-571ca1b37673,"[[2305.0, 2953.0]]"
5,0bb24bd2-b325-40ae-8de6-a60782494f72,"[[2248.9280555555556, 2340.244166666667]]"
6,0c0f4c97-aedd-42e6-9a3c-b5f628833c86,"[[497.48305555555555, 671.8]]"
7,0c31a588-734e-4641-b48c-cd46604c47a2,"[[2329.0, 3457.0]]"
8,0c7cfac7-5aac-457c-9d7b-c4b87685cb3f,"[[234.14305555555555, 1341.3944444444444]]"
9,0cf09d9d-2504-4989-ad68-62d41d151eff,"[[16.816666666666666, 28.825], [28.825, 53.033..."


In [107]:
import ast 

def extract_recovery(recovery: str):
    data = np.array(ast.literal_eval(recovery))
    if data.size == 2:
        data = data.reshape(2)
    
    return data

In [148]:
def recognize_limits(filename: str, recovery: list):
    if filename in ['1c9db047-e335-46ac-8039-effd8589b25b', '1cbce6e5-9f0b-419f-9527-7add4e255217']:
        pass
        return
    else:
        df = pd.read_csv(f'src/train_reduced/{filename}', sep = '\\s+')
    
    if recovery.size == 2:
        recovery_l, recovery_r = recovery[0], recovery[1]
        pressure_l = min(df[df['time'] >= recovery_l]['pressure'])
        pressure_r = min(df[df['time'] <= recovery_r]['pressure'])
        print(recovery_l, recovery_r)
        print(pressure_l, pressure_r)

        print('-' * 20)
    elif recovery.size > 2:
        for i in range(len(recovery)):
            recovery_l, recovery_r = recovery[i][0], recovery[i][1]
            pressure_l = min(df[df['time'] >= recovery_l]['pressure'])
            pressure_r = min(df[df['time'] <= recovery_r]['pressure'])
            print(recovery_l, recovery_r)
            print(pressure_l, pressure_r)
        print('-' * 20)

In [149]:
for idx in range(23):
    recovery = extract_recovery(label_df['recovery'][idx])
    recognize_limits(label_df['file'][idx], recovery)

2420.9805555555554 2438.4241666666667
78.712668 78.238427
3178.7994444444444 3454.6875
78.712668 78.238427
3764.9605555555554 3771.1033333333335
78.712668 78.238427
--------------------
10045.820833333333 10165.977777777778
37.645149 35.680339
--------------------
1172.6930555555555 1173.3916666666667
202.910941 203.291302
1268.9944444444445 1439.7258333333334
203.141287 202.910941
--------------------
2329.0 3457.0
44.493273 53.107038
--------------------
2305.0 2953.0
11.304806 111.216733
--------------------
2248.9280555555556 2340.244166666667
35.507175 39.378529
--------------------
497.48305555555555 671.8
15.302398 15.302398
--------------------
2329.0 3457.0
27.263191 35.963093
--------------------
234.14305555555555 1341.3944444444444
28.825226 37.342207
--------------------
16.816666666666666 28.825
188.404828 189.28169
28.825 53.033055555555556
188.404828 189.28169
53.033055555555556 108.49166666666666
188.404828 189.28169
--------------------
2343.0630555555554 3019.7725
56